# 日本 過去3年間の地震データ分析

このノートブックでは、米国地質調査所 (USGS) の FDSN Event API を利用して日本周辺 (緯度 24–46 度, 経度 122–154 度) で過去3年間に発生した地震データ (マグニチュード 2.5 以上) を取得し、日本の地震の総数、最大マグニチュードの件数、月次トレンドの分析と地図上で可視化します。

## 目的
1. 年別・月別発生件数の把握
2. マグニチュード分布と深さ分布の確認
3. 規模の大きい地震 (M 上位10件) の抽出
4. 月次推移トレンドの把握
5. 地図で可視化

In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta, timezone

# ---- 取得パラメータ設定 ----
JST = timezone(timedelta(hours=9))
now_utc = datetime.utcnow()
endtime = now_utc.strftime('%Y-%m-%d')  # USGSはUTC日付文字列
starttime = (now_utc - timedelta(days=365*3)).strftime('%Y-%m-%d')

# 日本周辺の概略範囲 (緯度経度)
minlatitude = 24
maxlatitude = 46
minlongitude = 122
maxlongitude = 154
minmagnitude = 2.5

# USGS FDSN Event API (CSV)
base_url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
params = {
    'format': 'csv',
    'starttime': starttime,
    'endtime': endtime,
    'minlatitude': minlatitude,
    'maxlatitude': maxlatitude,
    'minlongitude': minlongitude,
    'maxlongitude': maxlongitude,
    'minmagnitude': minmagnitude
}

print(f"データ取得期間 (UTC): {starttime} ～ {endtime}")
print("APIへリクエスト中...")
response = requests.get(base_url, params=params, timeout=60)
response.raise_for_status()

# 保存
csv_filename = 'earthquakes_japan_3y.csv'
with open(csv_filename, 'wb') as f:
    f.write(response.content)

print(f"保存しました: {csv_filename}, サイズ: {len(response.content)} bytes")

# 読み込み
raw_df = pd.read_csv(csv_filename)
print(f"取得行数: {len(raw_df)}")
raw_df.head()

C:\Users\kyuha\AppData\Local\Temp\ipykernel_15976\2108208179.py:7: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now_utc = datetime.utcnow()


データ取得期間 (UTC): 2022-11-13 ～ 2025-11-12
APIへリクエスト中...
保存しました: earthquakes_japan_3y.csv, サイズ: 696261 bytes
取得行数: 3737
保存しました: earthquakes_japan_3y.csv, サイズ: 696261 bytes
取得行数: 3737


,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2025-11-11T12:03:08.148Z,39.5501,143.1805,31.707,4.6,mb,47,103,2.040,0.63,...,2025-11-11T12:35:24.040Z,"106 km E of Yamada, Japan",earthquake,7.51,5.923,0.074,55,reviewed,us,us
1,2025-11-10T20:44:13.959Z,39.3160,143.6746,10.000,4.5,mb,46,136,2.486,0.69,...,2025-11-10T21:01:37.040Z,"149 km E of Yamada, Japan",earthquake,6.59,1.870,0.098,31,reviewed,us,us
2,2025-11-10T18:26:36.182Z,26.6744,130.2387,10.000,4.9,mb,73,95,1.765,0.85,...,2025-11-10T18:47:15.040Z,"170 km SE of Isen, Japan",earthquake,8.19,1.870,0.063,78,reviewed,us,us
3,2025-11-10T09:58:58.726Z,39.5209,143.2075,10.000,5.2,mb,61,133,2.074,0.80,...,2025-11-10T10:16:45.040Z,"108 km E of Yamada, Japan",earthquake,7.35,1.815,0.049,135,reviewed,us,us
4,2025-11-10T07:42:47.279Z,39.5397,143.5063,10.000,4.7,mb,73,144,2.249,0.57,...,2025-11-10T08:06:40.040Z,"134 km E of Yamada, Japan",earthquake,7.40,1.895,0.072,58,reviewed,us,us


In [ ]:
# 前処理と基礎統計 (軽量版)
import pandas as pd
import matplotlib.pyplot as plt

# raw_df が存在する前提。存在しない場合は読み込みを試行。
if 'raw_df' not in globals():
    raw_df = pd.read_csv('earthquakes_japan_3y.csv')

required_cols = ['time', 'latitude', 'longitude', 'depth', 'mag', 'place']
missing = [c for c in required_cols if c not in raw_df.columns]
if missing:
    raise ValueError(f"欠損列があります: {missing}")

raw_df['time'] = pd.to_datetime(raw_df['time'], errors='coerce')
raw_df = raw_df.dropna(subset=['time', 'mag'])
raw_df['year'] = raw_df['time'].dt.year
raw_df['month'] = raw_df['time'].dt.to_period('M')
raw_df = raw_df.sort_values('time').reset_index(drop=True)

# 年・月集計
year_counts = raw_df.groupby('year').size()
month_counts = raw_df.groupby('month').size()

print('形状:', raw_df.shape)
print('年別件数:\n', year_counts.to_string())
print('最新12か月件数:\n', month_counts.tail(12).to_string())

# 年別棒グラフ (簡易)
plt.figure(figsize=(6,3))
year_counts.plot(kind='bar', title='年別地震件数 (M>=2.5)')
plt.tight_layout()
plt.show()

raw_df.head(5)

In [ ]:
# マグニチュードと深さの分布 (日本語ラベル)
import matplotlib.pyplot as plt

plt.figure(figsize=(6,4))
raw_df['mag'].plot(kind='hist', bins=30, alpha=0.7, edgecolor='black')
plt.title('マグニチュード分布 (M≥2.5)')
plt.xlabel('マグニチュード')
plt.ylabel('度数')
plt.tight_layout()
plt.show()

plt.figure(figsize=(6,4))
raw_df['depth'].plot(kind='hist', bins=40, alpha=0.7, edgecolor='black', color='tab:orange')
plt.title('震源深さの分布')
plt.xlabel('深さ (km)')
plt.ylabel('度数')
plt.tight_layout()
plt.show()

print('深さの記述統計量:')
print(raw_df['depth'].describe())

In [ ]:
# Gutenberg-Richter b値推定 (日本語表示)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

magnitudes = raw_df['mag'].dropna()
Mcut = 3.5  # カタログ完全性を仮定する下限 (暫定)
subset = magnitudes[magnitudes >= Mcut]

if len(subset) < 10:
    raise ValueError('十分なデータがないため b値推定できません')

m_grid = np.arange(Mcut, subset.max() + 0.05, 0.1)
N = np.array([np.sum(subset >= m) for m in m_grid])
mask = N > 0
m_fit = m_grid[mask]
logN = np.log10(N[mask])

slope, intercept = np.polyfit(m_fit, logN, 1)
b_value = -slope

print(f"推定 b値 ≈ {b_value:.2f} (M ≥ {Mcut})")
print(f"回帰式: log10 N = {intercept:.2f} - {b_value:.2f} × M")

plt.figure(figsize=(6,4))
plt.scatter(m_fit, logN, s=25, label='観測値')
plt.plot(m_fit, slope*m_fit + intercept, 'r-', label=f'回帰直線 b={b_value:.2f}')
plt.xlabel('マグニチュード M')
plt.ylabel('log10 N(M≥M)')
plt.title('頻度-マグニチュード分布 (Gutenberg-Richter)')
plt.legend()
plt.tight_layout()
plt.show()

In [3]:
# 最大マグニチュード上位10件
largest10 = raw_df.sort_values('mag', ascending=False).head(10)[['time','latitude','longitude','depth','mag','place']]
print('最大マグニチュード上位10件:')
largest10.reset_index(drop=True)

最大マグニチュード上位10件:


,time,latitude,longitude,depth,mag,place
0,2024-01-01T07:10:09.476Z,37.4874,137.2710,10.0,7.5,"2024 Noto Peninsula, Japan Earthquake"
1,2024-08-08T07:42:55.206Z,31.7599,131.5021,24.0,7.1,"2024 Hyuganada Sea, Japan Earthquake"
2,2025-11-09T08:03:37.808Z,39.3957,143.4108,10.0,6.8,"126 km E of Yamada, Japan"
3,2025-01-13T12:19:32.252Z,31.8326,131.5525,39.0,6.8,"15 km SE of Miyazaki, Japan"
4,2023-12-28T09:15:16.131Z,44.5960,149.0388,31.0,6.5,"115 km SE of Kuril’sk, Russia"
5,2024-04-27T08:35:34.207Z,27.8209,139.6186,496.0,6.5,"Bonin Islands, Japan region"
6,2025-11-09T08:54:36.991Z,39.4662,143.3632,10.0,6.4,"121 km E of Yamada, Japan"
7,2023-01-16T04:49:51.935Z,28.9804,139.3452,405.0,6.3,"Bonin Islands, Japan region"
8,2024-04-17T14:14:46.694Z,33.2503,132.3647,32.0,6.3,"18 km W of Uwajima, Japan"
9,2023-09-18T13:21:23.535Z,26.5312,125.2312,176.0,6.3,"191 km N of Hirara, Japan"


In [ ]:
# 月次トレンド (件数推移) - 日本語ラベル
import pandas as pd
import matplotlib.pyplot as plt

work = raw_df.copy()
work['time'] = pd.to_datetime(work['time'], errors='coerce')
work = work.dropna(subset=['time'])
work['month'] = work['time'].dt.to_period('M')

monthly_series = work.groupby('month').size().sort_index()
monthly_df = monthly_series.to_frame(name='件数')
monthly_df.index = monthly_df.index.to_timestamp()

plt.figure(figsize=(10,4))
monthly_df['件数'].plot()
plt.title('月次地震件数の推移 (M≥2.5)')
plt.ylabel('件数')
plt.xlabel('月')
plt.tight_layout()
plt.show()

monthly_df.tail(12)

In [ ]:
# 集計結果の保存 (安全に再計算)
import pandas as pd

# 年別 / 月別 再計算
raw_df['time'] = pd.to_datetime(raw_df['time'], errors='coerce')
raw_df = raw_df.dropna(subset=['time'])
raw_df['year'] = raw_df['time'].dt.year
raw_df['month'] = raw_df['time'].dt.to_period('M')

year_counts = raw_df.groupby('year').size()
month_counts = raw_df.groupby('month').size()
monthly_df = month_counts.to_frame(name='count')

largest10 = raw_df.sort_values('mag', ascending=False).head(10)[['time','latitude','longitude','depth','mag','place']]

year_counts.to_csv('year_counts.csv', header=['count'])
month_counts.to_csv('month_counts.csv', header=['count'])
monthly_df.to_csv('monthly_trend.csv')
largest10.to_csv('largest10.csv', index=False)

print('保存完了:')
print(' - year_counts.csv')
print(' - month_counts.csv')
print(' - monthly_trend.csv')
print(' - largest10.csv')

In [1]:
# 日本周辺地震のマップ表示 (Plotly + Folium, JSTと深さ区分付き・拡大表示)
import pandas as pd
import plotly.express as px
import folium
from folium.plugins import MarkerCluster, Fullscreen

# データ読み込み (再実行対応)
if 'raw_df' not in globals():
    raw_df = pd.read_csv('earthquakes_japan_3y.csv')
    raw_df['time'] = pd.to_datetime(raw_df['time'], errors='coerce')

map_df = raw_df.dropna(subset=['latitude','longitude','mag','depth']).copy()
map_df['time'] = pd.to_datetime(map_df['time'], errors='coerce')

# JST列追加 (UTCとして扱ってから変換)
try:
    map_df['time_jst'] = map_df['time'].dt.tz_localize('UTC').dt.tz_convert('Asia/Tokyo')
except Exception:
    map_df['time_jst'] = map_df['time'] + pd.Timedelta(hours=9)

# 深さ区分関数
def depth_category(d):
    if d < 70:
        return '浅発 (0-70km)'
    elif d < 300:
        return '中深発 (70-300km)'
    else:
        return '深発 (300km~)'
map_df['深さ区分'] = map_df['depth'].apply(depth_category)

# 文字列化
time_fmt_jst = '%Y-%m-%d %H:%M JST'
time_fmt_utc = '%Y-%m-%d %H:%M UTC'
map_df['time_jst_str'] = map_df['time_jst'].dt.strftime(time_fmt_jst)
map_df['time_utc_str'] = map_df['time'].dt.strftime(time_fmt_utc)

# ---- Plotly (拡大サイズ) ----
fig = px.scatter_geo(
    map_df,
    lat='latitude', lon='longitude',
    color='深さ区分', size='mag',
    hover_name='place',
    hover_data={'mag':True,'depth':True,'time_utc_str':True,'time_jst_str':True},
    title='日本周辺 過去3年間 地震分布 (色: 深さ区分 / サイズ: M)',
    projection='natural earth'
)
fig.update_geos(lonaxis_range=[122,154], lataxis_range=[24,46], showcountries=True, countrycolor='gray')
fig.update_layout(legend_title_text='深さ区分', coloraxis_showscale=False,
                  width=1100, height=750, margin=dict(l=10,r=10,t=60,b=10))
fig.show()

# ---- Folium (全画面ボタン + 拡大表示) ----
center = [37.0, 137.0]
color_map = {
    '浅発 (0-70km)': 'blue',
    '中深発 (70-300km)': 'orange',
    '深発 (300km~)': 'purple'
}
subset = map_df[map_df['mag'] >= 4.5].copy()

m = folium.Map(location=center, zoom_start=6, tiles='CartoDB Positron', width='100%', height='700')
Fullscreen(position='topright').add_to(m)
cluster = MarkerCluster().add_to(m)
for _, r in subset.iterrows():
    popup = (f"<b>{r['place']}</b><br>UTC: {r['time_utc_str']}<br>JST: {r['time_jst_str']}<br>"
             f"M {r['mag']:.1f}, 深さ {r['depth']:.0f} km ({r['深さ区分']})")
    folium.CircleMarker(
        location=[r['latitude'], r['longitude']],
        radius=3 + float(r['mag']),
        color=color_map.get(r['深さ区分'],'gray'),
        fill=True,
        fill_color=color_map.get(r['深さ区分'],'gray'),
        fill_opacity=0.7,
        popup=popup
    ).add_to(cluster)

m  # Foliumオブジェクト表示

In [2]:
# FoliumマップHTML保存
m.save('earthquakes_map.html')
print('HTMLファイルを保存しました: earthquakes_map.html')

HTMLファイルを保存しました: earthquakes_map.html


## 感想
今回、USGS のオープンデータを用いて日本周辺の過去3年間の地震を集計・可視化し、数の多さや分布の偏りを実感しました。ヒストグラムからは中小規模の地震が日常的に発生していること、深さ分布からは沈み込み帯特有の深発地震が共存していることが読み取れます。b値がおよそ1.1という結果は、規模の小さい地震が相対的に多いという一般的傾向と整合的でした。月次の増減や地図上の集中域を眺めると、ニュースで見聞きする出来事の背後に、連続的で地域性のあるダイナミックな地球の活動があることを改めて感じます。数値やグラフは冷静ですが、その一つひとつの点は実際の揺れに結びついており、防災や備えを考える出発点にもなると考えました。